In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None

In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [3]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


521.509269952774

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [5]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


Реализуем UCB1, чтобы легче было оптимизировать.

In [2]:
def ucb(history):
    ucb.num_calls += 1
    ctr = history['clicks'] / (history['impressions'] + 1) #мне кажется, можно +1, а не +10
    xplr = np.sqrt(2 * np.log(ucb.num_calls) / (history['impressions'] + 1))
    idx = np.argmax(ctr + ucb.c * xplr)
    return history.index[idx]

In [4]:
for c in [0.001, 0.01, 0.1, 0.5, 1]:
    print(f"c = {c}")
    ucb.num_calls = 0
    ucb.c = c

    seed = 18475
    np.random.seed(seed=seed)

    start = time.time()
    output = simulation(ucb, n=200000, seed=seed)
    end = time.time()
    print(f"time cost: {end - start}")
    print(f"regret: {output['regret']}")
    print(f"mean regret: {output['regret']/output['rounds']}")
    print(f"total banners: {output['total_banners']} \n\n")

c = 0.001
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
time cost: 615.3238658905029
regret: 7570.250349143192
mean regret: 0.03785125174571596
total banners: 184 


c = 0.01
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simul

При c = 0.1 лучший результат, при с = 0.01 тоже проходит. Поробуем потюнить еще: 

In [5]:
for c in [0.04, 0.07, 0.25]:
    print(f"c = {c}")
    ucb.num_calls = 0
    ucb.c = c

    seed = 18475
    np.random.seed(seed=seed)

    start = time.time()
    output = simulation(ucb, n=200000, seed=seed)
    end = time.time()
    print(f"time cost: {end - start}")
    print(f"regret: {output['regret']}")
    print(f"mean regret: {output['regret']/output['rounds']}")
    print(f"total banners: {output['total_banners']} \n\n")

c = 0.04
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
time cost: 612.8775112628937
regret: 242.03395440174864
mean regret: 0.0012101697720087433
total banners: 184 


c = 0.07
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been sim

Лучший результат при c = 0.07:

regret: 152.6182167021829

mean regret: 0.0007630910835109144 